# Builtin A2A

NOTE: THIS IS EXPERIMENTAL AS OF WRTING OF THIS NOTEBOOK! THAT MEANS THE INTERFACE MAY HAVE BREAKING CHANGES IN THE FUTURE.

Last Update:

- July 17, 2025
- ADK == 1.6.1

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.sandbox.google.com/github/hupili/google-adk-101/blob/main/Builtin_A2A.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fhupili%2Fgoogle-adk-101%2Fmain%2FBuiltin_A2A.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-enterprise-logo.png" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://github.com/hupili/google-adk-101/blob/main/Builtin_A2A.ipynb">
      <img width="32px" src="https://www.gstatic.com/monospace/240802/git_host_github_mask.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/hupili/google-adk-101/main/Builtin_A2A.ipynb">
      <img width="32px" src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Environment Setup

In [2]:
import os

# API_KEY = '' # @param {type:"string"}
# os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE" # Use Vertex AI API
# os.environ["GOOGLE_API_KEY"] = API_KEY

PROJECT_ID = "hupili-genai-bb"  # @param {type:"string"}
if not PROJECT_ID:
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = "us-central1" # @param {type:"string"}

STAGING_BUCKET = 'gs://agent_engine_deploy_staging' # @param {type:"string"}

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE" # Use Vertex AI API

# [your-project-id]

In [3]:
import vertexai

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

In [4]:
from google.colab import auth
auth.authenticate_user()

NOTE: You may need to restart the session after installing adk

In [5]:
!pip install google-adk==1.6.1

## Utils and boilerplate

Below is our regular boilerplate used in this repo. Just a quick test if the basic ADK environment is working before we proceed to specific topics. You can simply run those codes and skip the details.

In [6]:
from google import adk

In [7]:
import json
import time

def pprint_events(events):
    '''Pretty print of events generated by ADK runner'''
    start_time = time.time()
    for _, event in enumerate(events):
        is_final_response = event.is_final_response()
        function_calls = event.get_function_calls()
        function_responses = event.get_function_responses()
        agent_res = json.loads(event.content.model_dump_json(indent=2, exclude_none=True))

        if is_final_response:
            print('>>> inside final response...')
            final_response = event.content.parts[0].text
            end_time = time.time()
            elapsed_time_ms = round((end_time - start_time) * 1000, 3)
            print(f'Final Response ({elapsed_time_ms} ms):\n{final_response}')
            print("-----------------------------")
        elif function_calls:
            print('+++ inside the function call...')
            for function_call in function_calls:
                print(f"function, [args]:  {function_call.name}, {function_call.args}")
        elif function_responses:
            print('--- inside the function call response...')
            # TODO(Pili): copied from walkthrough codes. Find root cause of 'pending' not found.
            # if not event.actions.pending:
            if True:
                for function_response in function_responses:
                    details = function_response.response
                    recommended_list = list(details.values())
                    print(f"Function Name: {function_response.name}")
                    result=json.dumps(recommended_list)
                    print(f"Function Results {result}")
            else:
                print(agent_res)
    print(f"Total elapsed time: {elapsed_time_ms}")


In [8]:
import random
import asyncio

from google.adk.tools import google_search
from google.adk import Agent
from google.adk.agents import Agent, LlmAgent
from google.genai import types
from pydantic import BaseModel
from google.genai import types
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

async def caller_factory(root_agent, app_name='App12345', user_id='User12345', session_id=None):
  session_service = InMemorySessionService()
  if session_id is None:
    suffix = random.randint(100000, 999999)
    session_id = f'{app_name}-{user_id}-{suffix}'
  session = await session_service.create_session(
      app_name=app_name, user_id=user_id, session_id=session_id)
  runner = Runner(agent=root_agent, app_name=app_name, session_service=session_service)
  def _call(query):
    content = types.Content(role='user', parts=[types.Part(text=query)])
    events = runner.run(user_id=session.user_id, session_id=session.id, new_message=content)
    return events
  return _call

In [9]:
MODEL = "gemini-2.5-flash"
APP_NAME = "simple_search_example"
USER_ID = "user12345"
SESSION_ID = "session12345"
AGENT_NAME = "simple_search_agent"

# Agent
simple_search_agent = Agent(
    model=MODEL,
    name="simple_search_agent",
    description="Agent to answer questions using Google Search.",
    instruction="I can answer your questions by searching the internet. Just ask me anything!",
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
    tools=[google_search],
)

In [10]:
call_agent = await caller_factory(simple_search_agent)

In [11]:
events = call_agent('Hello')

In [12]:
pprint_events(events)

>>> inside final response...
Final Response (1836.213 ms):
Hello! How can I help you today?
-----------------------------
Total elapsed time: 1836.213


In [13]:
events = call_agent('What is the weather in Hong Kong now?')
pprint_events(events)

>>> inside final response...
Final Response (4947.362 ms):
As of 12:18 PM HKT on Thursday, July 17, 2025, the weather in Hong Kong is cloudy with a temperature of 90 °F (32 °C). It feels like 105 °F (40 °C), and the humidity is around 70%. There is a 0% chance of rain currently.
-----------------------------
Total elapsed time: 4947.362


## Built-in A2A

In [14]:
!pip install google-adk[a2a]

INFO: pip is looking at multiple versions of grpcio-reflection to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.7/96.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.7 MB/s eta 0:00:00


### 1. A2A Server

- Put all your remote agents under `remote_agents`.
- The subdir name under this folder, will be served via `BASE/subdir/.well-known/agent.json` for A2A server discovery.


In [15]:
!mkdir -p remote_agents/search_agent

In [16]:
%%file remote_agents/search_agent/__init__.py

from . import agent

Writing remote_agents/search_agent/__init__.py


In [18]:
%%file remote_agents/search_agent/agent.py

from google.adk.tools import google_search
from google.adk import Agent
from google.adk.agents import Agent, LlmAgent
from google.genai import types
from pydantic import BaseModel
from google.genai import types
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

MODEL = "gemini-2.5-flash"

simple_search_agent = Agent(
    model=MODEL,
    name="simple_search_agent",
    description="Agent to answer questions using Google Search.",
    instruction="I can answer your questions by searching the internet. Just ask me anything!",
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
    tools=[google_search],
)

root_agent=simple_search_agent

Overwriting remote_agents/search_agent/agent.py


NOTE:

- The below agent card specifically tweaked a few names to show you which path really matters for the A2A server's end point.
- `localhost:8001/a2a/search_agent/.well-known/agent.json` -- In this URL, `search_agent` is the subdir under `remote_agents`, which is used in the CLI of `adk api_server`.
- Those do not matter: name in the Agent class, name in the agent.json, url in the agent.json

In [21]:
%%file remote_agents/search_agent/agent.json

{
  "capabilities": {},
  "defaultInputModes": ["text/plain"],
  "defaultOutputModes": ["text/plain"],
  "description": "Agent to help you do simple Google search and return a summary of the resutls.",
  "name": "simple_search_agent_on_a2a",
  "skills": [
    {
      "id": "simple_search",
      "name": "Simple Search",
      "description": "do simple Google search and return a summary of the resutls",
      "tags": ["intent-understanding", "summary", "google-search"]
    }
  ],
  "url": "http://localhost:8001/a2a/simple_search_agent_a2a_endpoint",
  "version": "1.0.0"
}

Overwriting remote_agents/search_agent/agent.json


In [22]:
!echo $GOOGLE_CLOUD_PROJECT

hupili-genai-bb


In [23]:
!echo $GOOGLE_CLOUD_LOCATION

us-central1


In [25]:
!gcloud auth application-default login


You are running on a Google Compute Engine virtual machine.
The service credentials associated with this virtual machine
will automatically be used by Application Default
Credentials, so it is not necessary to use this command.

If you decide to proceed anyway, your user credentials may be visible
to others with access to this virtual machine. Are you sure you want
to authenticate with your personal account?

Do you want to continue (Y/n)?  y

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=TotFPfmKcMANJjTuiCbhh0x4qvFUYY&prompt=consent&token_

In [26]:
!killall adk
!adk api_server --a2a --port 8001 remote_agents &> adk_api_server.log &

adk: no process found


In [34]:
!tail adk_api_server.log

/usr/local/lib/python3.11/dist-packages/google/adk/auth/credential_service/in_memory_credential_service.py:33: UserWarning: [EXPERIMENTAL] BaseCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__()
2025-07-17 04:19:51,901 - INFO - fast_api.py:1044 - Setting up A2A agent: search_agent
/usr/local/lib/python3.11/dist-packages/google/adk/cli/fast_api.py:1049: UserWarning: [EXPERIMENTAL] A2aAgentExecutor: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  agent_executor = A2aAgentExecutor(
2025-07-17 04:19:51,902 - INFO - fast_api.py:1075 - Successfully configured A2A agent: search_agent
INFO:     Started server process [2633]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8001 (Press CTRL+C to qui

In [35]:
!curl localhost:8001/a2a/search_agent/.well-known/agent.json

{"capabilities":{},"defaultInputModes":["text/plain"],"defaultOutputModes":["text/plain"],"description":"Agent to help you do simple Google search and return a summary of the resutls.","name":"simple_search_agent_on_a2a","protocolVersion":"0.2.5","skills":[{"description":"do simple Google search and return a summary of the resutls","id":"simple_search","name":"Simple Search","tags":["intent-understanding","summary","google-search"]}],"url":"http://127.0.0.1:8001/a2a/search_agent","version":"1.0.0"}

NOTE: if above agent card retrieval fails, please troubleshoot before proceed.

### 2. Test via A2A reference CLI

Note: this section is to show you the inner works of A2A from client side. If you just want to connect to A2AServer within ADK, you can skip this section.

Source repo:
https://github.com/a2aproject/a2a-python

In [36]:
!pip install a2a-sdk

In [37]:
import logging

from typing import Any
from uuid import uuid4

import httpx

from a2a.client import A2ACardResolver, A2AClient
from a2a.types import (
    AgentCard,
    MessageSendParams,
    SendMessageRequest,
    SendStreamingMessageRequest,
)

PUBLIC_AGENT_CARD_PATH = '/.well-known/agent.json'
EXTENDED_AGENT_CARD_PATH = '/agent/authenticatedExtendedCard'

# Configure logging to show INFO level messages
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)  # Get a logger instance

# --8<-- [start:A2ACardResolver]

base_url = 'http://localhost:8001/a2a/search_agent'

httpx_client = httpx.AsyncClient()

resolver = A2ACardResolver(
    httpx_client=httpx_client,
    base_url=base_url,
    # agent_card_path uses default, extended_agent_card_path also uses default
)

final_agent_card_to_use = await resolver.get_agent_card()

client = A2AClient(
    httpx_client=httpx_client, agent_card=final_agent_card_to_use
)

In [38]:
client

In [39]:
from pprint import pprint

#### Test batch request: Success

In [40]:
send_message_payload: dict[str, Any] = {
    'message': {
        'role': 'user',
        'parts': [
            {'kind': 'text', 'text': 'what is the current weather in Hong Kong?'}
        ],
        'messageId': uuid4().hex,
    },
}
request = SendMessageRequest(
    id=str(uuid4()), params=MessageSendParams(**send_message_payload)
)

response = await client.send_message(request)
pprint(response.model_dump(mode='json', exclude_none=True))
# --8<-- [end:send_message]

{'id': 'd5386b28-5bfa-4c8e-9456-621af6216be7',
 'jsonrpc': '2.0',
 'result': {'contextId': 'ca0ded9e-8411-4d79-aa9e-8dda9d07abae',
            'history': [{'contextId': 'ca0ded9e-8411-4d79-aa9e-8dda9d07abae',
                         'kind': 'message',
                         'messageId': '1df108a99acc47df9f7fbd317f8418d4',
                         'parts': [{'kind': 'text',
                                    'text': 'what is the current weather in '
                                            'Hong Kong?'}],
                         'role': 'user',
                         'taskId': '8a8573b0-1fbf-4b57-8268-0ed27681eec9'},
                        {'contextId': 'ca0ded9e-8411-4d79-aa9e-8dda9d07abae',
                         'kind': 'message',
                         'messageId': '1df108a99acc47df9f7fbd317f8418d4',
                         'parts': [{'kind': 'text',
                                    'text': 'what is the current weather in '
                                        

#### Test streaming: Error

There is SSE type error. We will circle back later. Check the ADK repo for a relevant issue (to be submitted soon).

In [41]:
# A2AClientHTTPError: HTTP Error 400: Invalid SSE response or protocol error: Expected response header Content-Type to contain 'text/event-stream', got 'application/json'

# # --8<-- [start:send_message_streaming]

# streaming_request = SendStreamingMessageRequest(
#     id=str(uuid4()), params=MessageSendParams(**send_message_payload)
# )

# stream_response = client.send_message_streaming(streaming_request)

# seq = 0
# async for chunk in stream_response:
#     seq += 1
#     print(f'=== seq: {seq} ===')
#     print(chunk.model_dump(mode='json', exclude_none=True))

# # --8<-- [end:send_message_streaming]

### A2A Client (in ADK)

NOTE: This CoLab is a quick way to show how ADK's built-in A2A works end to end, but not the intended way to build product. The client running within CoLab environment may encounter async event loop error every other message.

It is recommended to write the server and client parts locally and run in two separate shells.

In [42]:
# from google.adk.agents.remote_a2a_agent import AGENT_CARD_WELL_KNOWN_PATH
AGENT_CARD_WELL_KNOWN_PATH = "/.well-known/agent.json"
from google.adk.agents.remote_a2a_agent import RemoteA2aAgent

remote_search_agent = RemoteA2aAgent(
    name="remote_search_agent",
    description="Conduct searches and return a summary of the results",
    agent_card=(
        f"http://localhost:8001/a2a/search_agent{AGENT_CARD_WELL_KNOWN_PATH}"
    ),
)

root_agent = Agent(
    model=MODEL,
    name="root_agent",
    instruction="""
    Identify the user's intent
    - If user intends for an Internet search, delegate to remote_search_agent
    - Otherwise respond to the user by yourself.
    """,
    sub_agents=[remote_search_agent],
)

/tmp/ipython-input-42-901504012.py:5: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  remote_search_agent = RemoteA2aAgent(


In [43]:
call_agent = await caller_factory(root_agent)

In [44]:
events = call_agent('hello')

In [45]:
pprint_events(events)

/usr/local/lib/python3.11/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


>>> inside final response...
Final Response (2633.421 ms):
Hello! How can I help you today?
-----------------------------
Total elapsed time: 2633.421


In [46]:
events = call_agent('what is the weather in Hong Kong?')
# pprint_events(events)

In [47]:
events = list(events)

/usr/local/lib/python3.11/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/usr/local/lib/python3.11/dist-packages/google/adk/agents/remote_a2a_agent.py:346: UserWarning: [EXPERIMENTAL] convert_genai_part_to_a2a_part: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  converted_part = convert_genai_part_to_a2a_part(part)
/usr/local/lib/python3.11/dist-packages/google/adk/a2a/converters/event_converter.py:279: UserWarning: [EXPERIMENTAL] convert_a2a_message_to_event: This feature is experimental and may change or be removed in future versions without not

In [48]:
len(events)

3

In [49]:
events[0]

Event(content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'agent_name': 'remote_search_agent'
        },
        id='adk-2097afc5-fe92-4a38-afef-e73817c8b163',
        name='transfer_to_agent'
      )
    ),
  ],
  role='model'
), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=13,
  candidates_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=13
    ),
  ],
  prompt_token_count=249,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=249
    ),
  ],
  thoughts_token_count=84,
  total_token_count=346,
  traffic_type=<TrafficType.ON_DEMAND: 'ON_DEMAND'>
), invocation_id='e-b9934942-95d1-4402-9fbf-6f4fcea2edb2', author='root_agent', actions=EventActions(skip_summarizat

In [50]:
events[1]

Event(content=Content(
  parts=[
    Part(
      function_response=FunctionResponse(
        id='adk-2097afc5-fe92-4a38-afef-e73817c8b163',
        name='transfer_to_agent',
        response={
          'result': None
        }
      )
    ),
  ],
  role='user'
), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=None, invocation_id='e-b9934942-95d1-4402-9fbf-6f4fcea2edb2', author='root_agent', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent='remote_search_agent', escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='8394bca6-a49d-47bc-9a0b-65d1690e74b2', timestamp=1752726032.656509)

In [51]:
events[2]

Event(content=Content(
  parts=[
    Part(
      text="""The weather in Hong Kong is currently cloudy with a temperature of 90°F (32°C), feeling like 105°F (41°C). The humidity is around 70%, and there is a 0% chance of rain.

For today, Thursday, July 17, 2025, light rain is expected with a 25% chance of precipitation. Temperatures will range between 84°F (29°C) and 91°F (33°C), and humidity will be around 79%. The current time in Hong Kong is 12:20 PM.

Looking ahead, light rain, scattered thunderstorms, and rain are forecast for the upcoming days, with varying probabilities of precipitation and temperatures."""
    ),
  ],
  role='model'
), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata={'a2a:task_id': '9542e38e-3655-4b36-ba5e-d041b7bd45cf', 'a2a:context_id': '417907d8-3090-4c8a-9587-bac61b6fc07a', 'a2a:request': {'id': '8a5a0771-9b62-46de-a314-c5373d3cd8d3', 'jsonrpc': '2.0', 'method': 'message/send'

In [53]:
events = call_agent('what is the weather in Beijing?')
pprint_events(events)

/usr/local/lib/python3.11/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/usr/local/lib/python3.11/dist-packages/google/adk/agents/remote_a2a_agent.py:346: UserWarning: [EXPERIMENTAL] convert_genai_part_to_a2a_part: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  converted_part = convert_genai_part_to_a2a_part(part)


+++ inside the function call...
function, [args]:  transfer_to_agent, {'agent_name': 'remote_search_agent'}
--- inside the function call response...
Function Name: transfer_to_agent
Function Results [null]
>>> inside final response...
Final Response (8199.903 ms):
The current weather in Beijing, China is mostly sunny with a temperature of 93 °F (34 °C), feeling like 95 °F (35 °C). The humidity is around 34%, and there is a 0% chance of rain. The local time in Beijing is 12:21 PM.

The forecast for today, Thursday, July 17, 2025, is sunny during the day and clear with periodic clouds at night, with temperatures ranging from 76 °F (24 °C) to 97 °F (36 °C).

Here's a brief look at the upcoming days:
*   **Friday, July 18:** Mostly sunny during the day and partly cloudy at night, with a 10% chance of rain at night. Temperatures will be between 83 °F (28 °C) and 98 °F (37 °C).
*   **Saturday, July 19:** Light rain during the day and night, with a 35% chance of rain during the day and 40% at

/usr/local/lib/python3.11/dist-packages/google/adk/a2a/converters/event_converter.py:279: UserWarning: [EXPERIMENTAL] convert_a2a_message_to_event: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  return convert_a2a_message_to_event(message, author, invocation_context)
/usr/local/lib/python3.11/dist-packages/google/adk/a2a/converters/event_converter.py:345: UserWarning: [EXPERIMENTAL] convert_a2a_part_to_genai_part: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  part = convert_a2a_part_to_genai_part(a2a_part)


In [55]:
events = call_agent('is HK MOA open today?')
pprint_events(events)

/usr/local/lib/python3.11/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/usr/local/lib/python3.11/dist-packages/google/adk/agents/remote_a2a_agent.py:346: UserWarning: [EXPERIMENTAL] convert_genai_part_to_a2a_part: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  converted_part = convert_genai_part_to_a2a_part(part)


+++ inside the function call...
function, [args]:  transfer_to_agent, {'agent_name': 'remote_search_agent'}
--- inside the function call response...
Function Name: transfer_to_agent
Function Results [null]
>>> inside final response...
Final Response (8727.631 ms):
The Hong Kong Museum of Art (HKMoA) is closed on Thursdays, with the exception of public holidays. Since today is Thursday, July 17, 2025, and there is no indication of it being a public holiday, the HKMoA is likely closed.

The museum's general operating hours are:
*   **Mondays to Wednesdays, and Fridays:** 10:00 AM – 6:00 PM
*   **Saturdays, Sundays, and public holidays:** 10:00 AM – 9:00 PM
*   **Christmas Eve and Lunar New Year's Eve:** 10:00 AM – 5:00 PM
*   **Closed on:** Thursdays (except public holidays) and the first two days of the Lunar New Year.
-----------------------------
Total elapsed time: 8727.631


/usr/local/lib/python3.11/dist-packages/google/adk/a2a/converters/event_converter.py:279: UserWarning: [EXPERIMENTAL] convert_a2a_message_to_event: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  return convert_a2a_message_to_event(message, author, invocation_context)
/usr/local/lib/python3.11/dist-packages/google/adk/a2a/converters/event_converter.py:345: UserWarning: [EXPERIMENTAL] convert_a2a_part_to_genai_part: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  part = convert_a2a_part_to_genai_part(a2a_part)


In [56]:
events = call_agent('is HK MOA open today?')
# pprint_events(events)
events = list(events)

/usr/local/lib/python3.11/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [57]:
len(events)

1

In [58]:
events[0]

Event(content=Content(
  parts=[
    Part(
      text="Based on the information provided by the remote search agent, the Hong Kong Museum of Art (HKMoA) is closed today, Thursday, July 17, 2025, as Thursdays are their regular closing day (unless it's a public holiday, which is not indicated)."
    ),
  ],
  role='model'
), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=60,
  candidates_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=60
    ),
  ],
  prompt_token_count=1089,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=1089
    ),
  ],
  thoughts_token_count=139,
  total_token_count=1288,
  traffic_type=<TrafficType.ON_DEMAND: 'ON_DEMAND'>
), invocation_id='e-e9fd7aee-04c0-4446-8d92-e4ae8cb445a5', aut

In [59]:
events = call_agent('is Disney HK open today?')
# pprint_events(events)
events = list(events)

/usr/local/lib/python3.11/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/usr/local/lib/python3.11/dist-packages/google/adk/agents/remote_a2a_agent.py:346: UserWarning: [EXPERIMENTAL] convert_genai_part_to_a2a_part: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  converted_part = convert_genai_part_to_a2a_part(part)
ERROR:google_adk.google.adk.agents.remote_a2a_agent:A2A request failed: Event loop is closed


In [60]:
len(events)

3

In [61]:
events[0]

Event(content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'agent_name': 'remote_search_agent'
        },
        id='adk-e179c83a-c6fb-4458-8158-bcba438a9d8d',
        name='transfer_to_agent'
      )
    ),
  ],
  role='model'
), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=13,
  candidates_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=13
    ),
  ],
  prompt_token_count=1155,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=1155
    ),
  ],
  thoughts_token_count=80,
  total_token_count=1248,
  traffic_type=<TrafficType.ON_DEMAND: 'ON_DEMAND'>
), invocation_id='e-36882f6b-0413-4f68-b070-2c2a62fcc5df', author='root_agent', actions=EventActions(skip_summari

In [62]:
events[1]

Event(content=Content(
  parts=[
    Part(
      function_response=FunctionResponse(
        id='adk-e179c83a-c6fb-4458-8158-bcba438a9d8d',
        name='transfer_to_agent',
        response={
          'result': None
        }
      )
    ),
  ],
  role='user'
), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=None, invocation_id='e-36882f6b-0413-4f68-b070-2c2a62fcc5df', author='root_agent', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent='remote_search_agent', escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='6c374f65-9325-49d7-8bff-7654aa6ed809', timestamp=1752726135.38413)

In [63]:
events[2]

Event(content=None, grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message='A2A request failed: Event loop is closed', interrupted=None, custom_metadata={'a2a:request': {'id': '387e1545-2ab3-4577-822c-e1c8bf2ccb46', 'jsonrpc': '2.0', 'method': 'message/send', 'params': {'message': {'contextId': 'bea65661-2cb6-4e34-b497-8fe8c1e4bf1f', 'kind': 'message', 'messageId': '6b0deda4-26c6-4400-8c74-824a2a7f24db', 'parts': [{'kind': 'text', 'text': 'is HK MOA open today?'}, {'kind': 'text', 'text': "[root_agent] said: Based on the information provided by the remote search agent, the Hong Kong Museum of Art (HKMoA) is closed today, Thursday, July 17, 2025, as Thursdays are their regular closing day (unless it's a public holiday, which is not indicated)."}, {'kind': 'text', 'text': 'For context:'}, {'kind': 'text', 'text': 'is Disney HK open today?'}, {'kind': 'text', 'text': "[root_agent] called tool `transfer_to_agent` with parameters: {'agent_name': 'remote_se

In [64]:
events = call_agent('is Disney HK open today?')
# pprint_events(events)
events = list(events)

/usr/local/lib/python3.11/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/usr/local/lib/python3.11/dist-packages/google/adk/agents/remote_a2a_agent.py:346: UserWarning: [EXPERIMENTAL] convert_genai_part_to_a2a_part: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  converted_part = convert_genai_part_to_a2a_part(part)
/usr/local/lib/python3.11/dist-packages/google/adk/a2a/converters/event_converter.py:279: UserWarning: [EXPERIMENTAL] convert_a2a_message_to_event: This feature is experimental and may change or be removed in future versions without not

In [65]:
len(events)

3

In [66]:
events[2]

Event(content=Content(
  parts=[
    Part(
      text="Yes, Hong Kong Disneyland is open today, Thursday, July 17, 2025. The park's operating hours are from 10:00 AM to 9:00 PM."
    ),
  ],
  role='model'
), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata={'a2a:task_id': '572e9409-27db-46b8-9fbd-1adf84917b18', 'a2a:context_id': '94e7c34b-173f-4c0b-aa19-05f0b1be5b0e', 'a2a:request': {'id': '69a83a44-bf4d-47f8-9386-bd5a146a7d74', 'jsonrpc': '2.0', 'method': 'message/send', 'params': {'message': {'kind': 'message', 'messageId': '74d61c8f-8d35-4fe6-adf1-287d01673620', 'parts': [{'kind': 'text', 'text': 'is Disney HK open today?'}, {'kind': 'text', 'text': "[root_agent] called tool `transfer_to_agent` with parameters: {'agent_name': 'remote_search_agent'}"}, {'kind': 'text', 'text': 'For context:'}, {'kind': 'text', 'text': "[root_agent] `transfer_to_agent` tool returned result: {'result': None}"}, {'kind': '